### Libraries

In [1]:
import glob
import os
import pandas as pd

___
### Выгрузка данных 

In [4]:
survey_fcasts = pd.DataFrame()

file_type = 'csv'
seperator =','

for f in glob.glob("*." + file_type):
    if f[:13] == 'survey_fcasts':
        data = pd.read_csv(f, sep=seperator, low_memory=False, encoding='ISO-8859-1')
        survey_fcasts = pd.concat([survey_fcasts, data])
    

ifps = pd.read_csv('ifps.csv', sep=seperator, encoding='ISO-8859-1')

In [5]:
survey_fcasts.head(5)

,ifp_id,ctt,cond,training,team,user_id,forecast_id,fcast_type,answer_option,value,fcast_date,expertise,q_status,viewtime,year,timestamp
0,1004-0,1a,1,a,NaN,600.0,-200987.0,0,a,0.10,2011-08-31,1.0,closed,NaN,1,2011-08-31 16:17:18
1,1004-0,1a,1,a,NaN,600.0,-200987.0,0,b,0.90,2011-08-31,1.0,closed,NaN,1,2011-08-31 16:17:18
2,1008-0,1a,1,a,NaN,600.0,-200986.0,0,a,0.25,2011-08-31,2.0,closed,NaN,1,2011-08-31 16:18:27
3,1008-0,1a,1,a,NaN,600.0,-200986.0,0,b,0.75,2011-08-31,2.0,closed,NaN,1,2011-08-31 16:18:27
4,1006-0,1a,1,a,NaN,600.0,-200985.0,0,a,0.10,2011-08-31,3.0,closed,NaN,1,2011-08-31 16:18:47


In [6]:
ifps.head(5)

,ifp_id,q_type,q_text,q_desc,q_status,date_start,date_suspend,date_to_close,date_closed,outcome,short_title,days_open,n_opts,options
0,1001-0,0,"Will the Six-Party talks (among the US, North ...",'In' refers to any time during the remainder o...,closed,9/1/11,12/30/11 0:00,12/31/11,1/2/12,b,Six-party talks resume,123.0,2,"(a) Yes, (b) No"
1,1002-0,0,Who will be inaugurated as President of Russia...,'In' refers to any time during the 2012 calend...,closed,9/1/11,5/14/12 0:00,5/15/12,5/6/12,b,president of Russia,248.0,3,"(a) Medvedev, (b) Putin, (c) Neither"
2,1003-0,0,Will Serbia be officially granted EU candidacy...,A 'yes' answer to this question requires not o...,closed,9/1/11,12/30/11 0:00,12/31/11,1/3/12,b,Serbia EU candidacy,124.0,2,"(a) Yes, (b) No"
3,1004-0,0,Will the United Nations General Assembly recog...,'By' means at or prior to the end of the day o...,closed,9/1/11,9/29/11 0:00,9/30/11,9/30/11,b,UN-GA recognize Palestine,29.0,2,"(a) Yes, (b) No"
4,1005-0,0,Will Daniel Ortega win another term as Preside...,If the Nicaraguan elections do not occur in la...,closed,9/1/11,11/4/11 0:00,11/5/11,11/5/11,a,Ortega win in Nicaragua,65.0,2,"(a) Yes, (b) No"


In [7]:
q_ifps = pd.DataFrame(ifps['q_text'].unique(), columns=['questions'])
unique_q_num = q_ifps.nunique()

# show some of them
(
q_ifps.questions[0], 
q_ifps.questions[391], 
q_ifps.questions[200] 
)

('Will the Six-Party talks (among the US, North Korea, South Korea, Russia, China, and Japan) formally resume in 2011?',
 'Will **national military force(s) from one or more countries *intervene in Syria **before 1 December 2014?',
 'When will Mariano Rajoy vacate the office of Prime Minister of Spain?')

In [8]:
f'Всего вопросов : {unique_q_num[0]}'

'Всего вопросов : 527'

___
### Предсказание людей в зависимости от их ответа в прошлый год

In [38]:
user_answers = survey_fcasts[['ifp_id','user_id','q_status','year','answer_option','timestamp','value','fcast_type']]
user_answers = user_answers[(~user_answers.user_id.isna()) & (user_answers.q_status == 'closed')].reset_index(drop=True)

user_answers['ans_type'] = user_answers.ifp_id.apply(lambda x: x[5:]).astype(int)
user_answers['question'] = user_answers.ifp_id.apply(lambda x: x[:4]).astype(int)

assert user_answers.user_id.apply(lambda x: x.is_integer()).sum() == user_answers.shape[0]

user_answers['user_id'] = user_answers['user_id'].astype(int)
user_answers = user_answers.sort_values(['question','user_id'])

In [39]:
user_answers.head(2)

,ifp_id,user_id,q_status,year,answer_option,timestamp,value,fcast_type,ans_type,question
5555,1001-0,3,closed,1,a,2011-09-06 05:46:12,0.5,1,0,1001
5556,1001-0,3,closed,1,b,2011-09-06 05:46:12,0.5,1,0,1001


In [40]:
mask = (ifps.q_status == 'closed') & (ifps.n_opts == 2)
question_answers = ifps[mask][['ifp_id','outcome']]
assert question_answers.ifp_id.nunique() == question_answers.shape[0]

In [41]:
question_answers.head(2)

,ifp_id,outcome
0,1001-0,b
2,1003-0,b


In [42]:
ans = question_answers.merge(user_answers, on='ifp_id', how='left').drop(['q_status', 'ans_type', 'question','fcast_type'], axis=1)
ans.timestamp = ans.timestamp.astype('datetime64[ns]')

temp = ans.groupby(['ifp_id','user_id']).timestamp.max().reset_index()
ans = temp.merge(ans, how='left', on=['user_id','ifp_id','timestamp'])

In [43]:
ans[(ans.ifp_id == '1047-0') & (ans.user_id == 5085)]

,ifp_id,user_id,timestamp,outcome,year,answer_option,value
94548,1047-0,5085,2012-08-08 11:43:11,b,2,a,0.23
94549,1047-0,5085,2012-08-08 11:43:11,b,2,b,0.77
94550,1047-0,5085,2012-08-08 11:43:11,b,2,a,0.23
94551,1047-0,5085,2012-08-08 11:43:11,b,2,b,0.77
94552,1047-0,5085,2012-08-08 11:43:11,b,2,a,0.23
94553,1047-0,5085,2012-08-08 11:43:11,b,2,b,0.77
94554,1047-0,5085,2012-08-08 11:43:11,b,2,a,0.23
94555,1047-0,5085,2012-08-08 11:43:11,b,2,b,0.77
94556,1047-0,5085,2012-08-08 11:43:11,b,2,a,0.23
94557,1047-0,5085,2012-08-08 11:43:11,b,2,b,0.77


In [44]:
## иначе на одну дату много вариантов ответов
ans = ans.drop_duplicates()

## единственный пример где больше двух вариантов после всех упрощений 
#ans_new[(ans_new.ifp_id == '1047-0') & (ans_new.user_id == 5085)]

temp = ans.groupby(['ifp_id','user_id']).value.max().reset_index()
ans = temp.merge(ans, how='left', on=['user_id','ifp_id','value'])

In [45]:
ans[(ans.user_id == 3) & (ans.ifp_id == '1001-0')]

,ifp_id,user_id,value,timestamp,outcome,year,answer_option
0,1001-0,3,0.5,2011-09-06 05:46:12,b,1,a
1,1001-0,3,0.5,2011-09-06 05:46:12,b,1,b


In [46]:
## чтобы выбрать один вариант где 50% вероятность ответа
ans = ans.drop_duplicates(['ifp_id','user_id','value']).drop(['value','timestamp'], axis=1)


## оставим только тех кто больше чем в 1 год отвечал
temp = ans.groupby('user_id').year.nunique().reset_index()
ids_more_1_y = temp[temp.year > 1].user_id.unique()

ans = ans[ans.user_id.isin(ids_more_1_y)]
ans['correct_answer'] = (ans.outcome == ans.answer_option).astype('int8')

In [50]:
ans.head(3)

,ifp_id,user_id,outcome,year,answer_option,correct_answer
2,1001-0,6,b,1,b,1
8,1001-0,35,b,1,a,0
14,1001-0,47,b,1,b,1


In [53]:
ans.groupby(['user_id','year']).agg({'ifp_id':'count','correct_answer':'mean'}).reset_index()

,user_id,year,ifp_id,correct_answer
0,6,1,68,0.867647
1,6,2,23,0.782609
2,6,3,18,0.888889
3,35,1,27,0.629630
4,35,2,35,0.571429
...,...,...,...,...
3407,15836,4,81,0.975309
3408,15840,3,49,0.979592
3409,15840,4,5,0.400000
3410,15863,3,20,0.950000
